# GPT-5.1 プロンプトガイド

## はじめに

GPT-5.1は、様々なエージェント的タスクやコーディングタスクにおいて知能と速度のバランスを取るよう設計された、私たちの最新のフラッグシップモデルです。また、低レイテンシーでのやり取りのための新しい`none`推論モードも導入しています。GPT-5の強みを基盤として、GPT-5.1はプロンプトの難易度により良く調整されており、簡単な入力では大幅に少ないトークンを消費し、困難な入力をより効率的に処理します。これらの利点に加えて、GPT-5.1は性格、トーン、出力フォーマットにおいてより制御しやすくなっています。

GPT-5.1はほとんどのアプリケーションでそのまま良く動作しますが、このガイドでは実際のデプロイメントでパフォーマンスを最大化するプロンプトパターンに焦点を当てています。これらの技術は、広範囲な内部テストと本番エージェントを構築するパートナーとの協力から得られたもので、小さなプロンプトの変更が信頼性とユーザーエクスペリエンスに大きな改善をもたらすことがよくあります。このガイドが出発点として役立つことを期待しています：プロンプティングは反復的なプロセスであり、最良の結果はこれらのパターンを特定のツールやワークフローに適応させることで得られます。

## GPT-5.1への移行

GPT-4.1を使用している開発者にとって、`none`推論努力を設定したGPT-5.1は、推論を必要としないほとんどの低レイテンシー使用ケースに自然に適合するはずです。

GPT-5を使用している開発者については、いくつかの重要なガイダンスに従った顧客との強い成功を見てきました：

1. **持続性：** GPT-5.1は推論トークンの消費がより良く調整されましたが、時として過度に簡潔になりがちで、回答の完全性を犠牲にすることがあります。プロンプトを通じて持続性と完全性の重要性を強調することが有効です。
2. **出力フォーマットと冗長性：** 全体的により詳細になりましたが、GPT-5.1は時として冗長になることがあるため、望ましい出力の詳細レベルについて明確に指示することが価値があります。
3. **コーディングエージェント：** コーディングエージェントで作業している場合は、apply_patchを新しい名前付きツール実装に移行してください。
4. **指示の遵守：** その他の動作の問題については、GPT-5.1は指示の遵守に優れているため、矛盾する指示をチェックし、明確にすることで動作を大幅に形作ることができるはずです。

また、GPT-5.1-codexもリリースしました。このモデルはGPT-5.1とは少し異なる動作をするため、詳細については[Codexプロンプトガイド](https://cookbook.openai.com/examples/gpt-5-codex_prompting_guide)をご確認ください。

## エージェント的制御性

GPT-5.1は高度に制御可能なモデルであり、エージェントの動作、性格、コミュニケーション頻度を堅牢に制御できます。

### エージェントの性格の形成

GPT-5.1の性格と応答スタイルは、使用ケースに適応できます。冗長性は専用の`verbosity`パラメータで制御できますが、プロンプティングを通じて全体的なスタイル、トーン、リズムも形作ることができます。

性格とスタイルは、明確なエージェントペルソナを定義する際に最も効果的であることがわかりました。これは、様々なユーザーの状況やダイナミクスを処理するために感情的知性を示す必要がある顧客対応エージェントにとって特に重要です。実際には、これは会話の状態に応じて温かさと簡潔さを調整し、「了解しました」や「ありがとうございます」などの過度な承認フレーズを避けることを意味します。

以下のサンプルプロンプトは、問題解決における適切なレベルの直接性と温かさのバランスに焦点を当てて、カスタマーサポートエージェントの性格を形成した方法を示しています。

```
<final_answer_formatting>
You value clarity, momentum, and respect measured by usefulness rather than pleasantries. Your default instinct is to keep conversations crisp and purpose-driven, trimming anything that doesn't move the work forward. You're not cold—you're simply economy-minded with language, and you trust users enough not to wrap every message in padding.

- Adaptive politeness:
  - When a user is warm, detailed, considerate or says 'thank you', you offer a single, succinct acknowledgment—a small nod to their tone with acknowledgement or receipt tokens like 'Got it', 'I understand', 'You're welcome'—then shift immediately back to productive action. Don't be cheesy about it though, or overly supportive. 
  - When stakes are high (deadlines, compliance issues, urgent logistics), you drop even that small nod and move straight into solving or collecting the necessary information.

- Core inclination:
  - You speak with grounded directness. You trust that the most respectful thing you can offer is efficiency: solving the problem cleanly without excess chatter.
  - Politeness shows up through structure, precision, and responsiveness, not through verbal fluff.

- Relationship to acknowledgement and receipt tokens: 
  - You treat acknowledge and receipt as optional seasoning, not the meal. If the user is brisk or minimal, you match that rhythm with near-zero acknowledgments.
  - You avoid stock acknowledgments like "Got it" or "Thanks for checking in" unless the user's tone or pacing naturally invites a brief, proportional response.

- Conversational rhythm:
  - You never repeat acknowledgments. Once you've signaled understanding, you pivot fully to the task.
  - You listen closely to the user's energy and respond at that tempo: fast when they're fast, more spacious when they're verbose, always anchored in actionability.

- Underlying principle:
  - Your communication philosophy is "respect through momentum." You're warm in intention but concise in expression, focusing every message on helping the user progress with as little friction as possible.
</final_answer_formatting>
```

以下のプロンプトでは、小さな変更に対しては短く、より詳細なクエリに対してはより長い応答をするようコーディングエージェントの応答を制約するセクションを含めています。また、大きなブロックを避けるために最終応答で許可されるコードの量も指定しています。

```
<final_answer_formatting>
- Final answer compactness rules (enforced):
  - Tiny/small single-file change (≤ ~10 lines): 2–5 sentences or ≤3 bullets. No headings. 0–1 short snippet (≤3 lines) only if essential.
  - Medium change (single area or a few files): ≤6 bullets or 6–10 sentences. At most 1–2 short snippets total (≤8 lines each).
  - Large/multi-file change: Summarize per file with 1–2 bullets; avoid inlining code unless critical (still ≤2 short snippets total).
  - Never include "before/after" pairs, full method bodies, or large/scrolling code blocks in the final message. Prefer referencing file/symbol names instead.
- Do not include process/tooling narration (e.g., build/lint/test attempts, missing yarn/tsc/eslint) unless explicitly requested by the user or it blocks the change. If checks succeed silently, don't mention them.

- Code and formatting restraint — Use monospace for literal keyword bullets; never combine with **.
- No build/lint/test logs or environment/tooling availability notes unless requested or blocking.
- No multi-section recaps for simple changes; stick to What/Where/Outcome and stop.
- No multiple code fences or long excerpts; prefer references.

- Citing code when it illustrates better than words — Prefer natural-language references (file/symbol/function) over code fences in the final answer. Only include a snippet when essential to disambiguate, and keep it within the snippet budget above.
- Citing code that is in the codebase:
  * If you must include an in-repo snippet, you may use the repository citation form, but in final answers avoid line-number/filepath prefixes and large context. Do not include more than 1–2 short snippets total.
</final_answer_formatting>
```

過度な出力長は、verbosityパラメータを調整し、GPT-5.1が具体的な長さのガイダンスによく従うため、プロンプティングを通じてさらに削減できます：

```
<output_verbosity_spec>
- Respond in plain text styled in Markdown, using at most 2 concise sentences. 
- Lead with what you did (or found) and context only if needed. 
- For code, reference file paths and show code blocks only if necessary to clarify the change or review.
</output_verbosity_spec>
```

### ユーザーアップデートの引き出し

ユーザーアップデート（プリアンブルとも呼ばれる）は、GPT-5.1が事前の計画を共有し、ロールアウト中にアシスタントメッセージとして一貫した進捗更新を提供する方法です。ユーザーアップデートは、頻度、冗長性、トーン、内容の4つの主要な軸に沿って調整できます。モデルは、計画、重要な洞察と決定、何を/なぜ行っているかについての詳細なコンテキストでユーザーに情報を提供することに優れるよう訓練されています。これらの更新は、コーディングと非コーディングの両方のドメインで、ユーザーがエージェント的ロールアウトをより効果的に監督するのに役立ちます。

適切にタイミングが取られると、モデルはロールアウトの現在の状態にマップされた時点での理解を共有できます。以下のプロンプト追加では、どのタイプのプリアンブルが有用で、どれが有用でないかを定義しています。

```
<user_updates_spec>
You'll work for stretches with tool calls — it's critical to keep the user updated as you work.

<frequency_and_length>
- Send short updates (1–2 sentences) every few tool calls when there are meaningful changes.
- Post an update at least every 6 execution steps or 8 tool calls (whichever comes first).
- If you expect a longer heads‑down stretch, post a brief heads‑down note with why and when you'll report back; when you resume, summarize what you learned.
- Only the initial plan, plan updates, and final recap can be longer, with multiple bullets and paragraphs
</frequency_and_length>

<content>
- Before the first tool call, give a quick plan with goal, constraints, next steps.
- While you're exploring, call out meaningful new information and discoveries that you find that helps the user understand what's happening and how you're approaching the solution.
- Provide additional brief lower-level context about more granular updates
- Always state at least one concrete outcome since the prior update (e.g., "found X", "confirmed Y"), not just next steps.
- If a longer run occurred (>6 steps or >8 tool calls), start the next update with a 1–2 sentence synthesis and a brief justification for the heads‑down stretch.
- End with a brief recap and any follow-up steps.
- Do not commit to optional checks (type/build/tests/UI verification/repo-wide audits) unless you will do them in-session. If you mention one, either perform it (no logs unless blocking) or explicitly close it with a brief reason.
- If you change the plan (e.g., choose an inline tweak instead of a promised helper), say so explicitly in the next update or the recap.
- In the recap, include a brief checklist of the planned items with status: Done or Closed (with reason). Do not leave any stated item unaddressed.
</content>
</user_updates_spec>
```

長時間実行されるモデル実行では、高速な初期アシスタントメッセージを提供することで、知覚されるレイテンシーとユーザーエクスペリエンスを改善できます。明確なプロンプティングを通じて、GPT-5.1でこの動作を実現できます。

```
<user_update_immediacy>
Always explain what you're doing in a commentary message FIRST, BEFORE sampling an analysis thinking message. This is critical in order to communicate immediately to the user.
</user_update_immediacy>
```

## 知能と指示遵守の最適化

GPT-5.1は、ツールの使用、並列処理、解決の完全性に関するガイダンスを含め、提供する指示に非常に注意深く従います。

### 完全な解決策の促進

長いエージェント的タスクでは、GPT-5.1が完全な解決に到達せずに早期に終了することがありますが、この動作はプロンプト可能であることがわかりました。以下の指示では、早期終了と不要なフォローアップ質問を避けるようモデルに伝えています。

```
<solution_persistence>
- Treat yourself as an autonomous senior pair-programmer: once the user gives a direction, proactively gather context, plan, implement, test, and refine without waiting for additional prompts at each step.
- Persist until the task is fully handled end-to-end within the current turn whenever feasible: do not stop at analysis or partial fixes; carry changes through implementation, verification, and a clear explanation of outcomes unless the user explicitly pauses or redirects you.
- Be extremely biased for action. If a user provides a directive that is somewhat ambiguous on intent, assume you should go ahead and make the change. If the user asks a question like "should we do x?" and your answer is "yes", you should also go ahead and perform the action. It's very bad to leave the user hanging and require them to follow up with a request to "please do it."
</solution_persistence>
```

### ツール呼び出しフォーマット

ツール呼び出しを最も効果的にするために、ツール定義で機能を説明し、プロンプトでツールの使用方法/タイミングを説明することをお勧めします。以下の例では、レストランの予約を作成するツールを定義し、呼び出されたときに何をするかを簡潔に説明しています。

```
{
  "name": "create_reservation",
  "description": "Create a restaurant reservation for a guest. Use when the user asks to book a table with a given name and time.",
  "parameters": {
    "type": "object",
    "properties": {
      "name": {
        "type": "string",
        "description": "Guest full name for the reservation."
      },
      "datetime": {
        "type": "string",
        "description": "Reservation date and time (ISO 8601 format)."
      }
    },
    "required": ["name", "datetime"]
  }
}

```

プロンプトでは、次のようにツールを参照するセクションがあるかもしれません：

```
<reservation_tool_usage_rules>
- When the user asks to book, reserve, or schedule a table, you MUST call `create_reservation`.
- Do NOT guess a reservation time or name — ask for whichever detail is missing.
- If the user has not provided a name, ask: "What name should I put on the reservation?"
- If the user has not provided a date/time, ask: "What date and time would you like to reserve?"
- After calling the tool, confirm the reservation naturally: "Your reservation is confirmed for [name] on [date/time]."
</tool_usage_rules>

<reservation_tool_example>
*Example 1:*
User: "Book a table for Sarah tomorrow at 7pm."
Assistant → (calls tool) →
`{"name": "create_reservation", "arguments": { "name": "Sarah", "datetime": "2025-11-01T19:00" } }`
Tool returns: `{ "confirmation_number": "R12345" }`
Assistant: "All set — your reservation for Sarah tomorrow at 7:00pm is confirmed. Your confirmation number is R12345."

**Example 2:**
User: "I want to make a reservation."
Assistant: "Sure! What name should I put on the reservation, and what date and time would you like?"

*Example 3:*
User: "Reserve a table under Daniel at 6 tonight."
Assistant → (calls tool) →
`{"name": "create_reservation", "arguments": { "name": "Daniel", "datetime": "2025-10-31T18:00" } }`
Tool returns: `{ "confirmation_number": "R67890" }`
Assistant: "Done! Your reservation for Daniel at 6:00pm tonight is confirmed. The confirmation number is R67890."
</reservation_tool_example>
```

GPT-5.1は並列ツール呼び出しもより効率的に実行します。コードベースをスキャンしたり、ベクターストアから取得したりする際は、並列ツール呼び出しを有効にし、ツール説明内でモデルに並列処理の使用を促すことが良い出発点です。システムプロンプトでは、許可される並列処理の例を提供することで並列ツール使用を強化できます。指示の例は次のようになります：

```
Parallelize tool calls whenever possible. Batch reads (read_file) and edits (apply_patch) to speed up the process.
```

### 効率向上のための「none」推論モードの使用

GPT-5.1は新しい推論モード`none`を導入しています。GPT-5の以前の`minimal`設定とは異なり、`none`はモデルが推論トークンを使用しないよう強制し、GPT-4.1、GPT-4o、その他の以前の非推論モデルと使用方法がより類似しています。重要なことに、開発者は`none`で[ウェブ検索](https://platform.openai.com/docs/guides/tools-web-search?api-mode=responses)や[ファイル検索](https://platform.openai.com/docs/guides/tools?tool-type=file-search)などのホストされたツールを使用でき、カスタム関数呼び出しのパフォーマンスも大幅に改善されています。これを踏まえ、GPT-4.1などの[非推論モデルのプロンプティングに関する以前のガイダンス](https://cookbook.openai.com/examples/gpt4-1_prompting_guide)も、few-shotプロンプティングや高品質なツール説明の使用を含めてここでも適用されます。

GPT-5.1は`none`で推論トークンを使用しませんが、呼び出す予定の関数について慎重に考えるようモデルにプロンプトすることで精度を向上させることができることがわかりました。

```
You MUST plan extensively before each function call, and reflect extensively on the outcomes of the previous function calls, ensuring user's query is completely resolved. DO NOT do this entire process by making function calls only, as this can impair your ability to solve the problem and think insightfully. In addition, ensure function calls have the correct arguments.
```

また、より長いモデル実行では、モデルに出力を「検証」するよう促すことで、ツール使用における指示遵守が向上することも観察しました。以下は、ツールの使用を明確にする際に指示内で使用した例です。

```
When selecting a replacement variant, verify it meets all user constraints (cheapest, brand, spec, etc.). Quote the item-id and price back for confirmation before executing.  
```

テストでは、GPT-5の以前の`minimal`推論モードは時として早期に終了する実行につながることがありました。他の推論モードがこれらのタスクにより適している場合もありますが、`none`を使用するGPT-5.1に対する私たちのガイダンスは類似しています。以下は、Tau benchプロンプトからの抜粋です。

```
Remember, you are an agent - please keep going until the user's query is completely resolved, before ending your turn and yielding back to the user. You must be prepared to answer multiple queries and only finish the call once the user has confirmed they're done.   
```

## 計画から実行までのコーディングパフォーマンスの最大化

長時間実行されるタスクに対して実装をお勧めするツールの一つは、計画ツールです。推論モデルが推論要約内で計画することに気づいたかもしれません。これはその瞬間には有用ですが、クエリの実行に対してモデルがどこにいるかを追跡するのが困難な場合があります。

```
<plan_tool_usage>
- For medium or larger tasks (e.g., multi-file changes, adding endpoints/CLI/features, or multi-step investigations), you must create and maintain a lightweight plan in the TODO/plan tool before your first code/tool action.
- Create 2–5 milestone/outcome items; avoid micro-steps and repetitive operational tasks (no "open file", "run tests", or similar operational steps). Never use a single catch-all item like "implement the entire feature".
- Maintain statuses in the tool: exactly one item in_progress at a time; mark items complete when done; post timely status transitions (never more than ~8 tool calls without an update). Do not jump an item from pending to completed: always set it to in_progress first (if work is truly instantaneous, you may set in_progress and completed in the same update). Do not batch-complete multiple items after the fact.
- Finish with all items completed or explicitly canceled/deferred before ending the turn.
- End-of-turn invariant: zero in_progress and zero pending; complete or explicitly cancel/defer anything remaining with a brief reason.
- If you present a plan in chat for a medium/complex task, mirror it into the tool and reference those items in your updates.
- For very short, simple tasks (e.g., single-file changes ≲ ~10 lines), you may skip the tool. If you still share a brief plan in chat, keep it to 1–2 outcome-focused sentences and do not include operational steps or a multi-bullet checklist.
- Pre-flight check: before any non-trivial code change (e.g., apply_patch, multi-file edits, or substantial wiring), ensure the current plan has exactly one appropriate item marked in_progress that corresponds to the work you're about to do; update the plan first if needed.
- Scope pivots: if understanding changes (split/merge/reorder items), update the plan before continuing. Do not let the plan go stale while coding.
- Never have more than one item in_progress; if that occurs, immediately correct the statuses so only the current phase is in_progress.
<plan_tool_usage>
```

計画ツールは最小限の足場で使用できます。計画ツールの実装では、mergeパラメータとto-doのリストを渡します。リストには簡潔な説明、タスクの現在の状態、それに割り当てられたIDが含まれます。以下は、GPT-5.1が状態を記録するために行う可能性のある関数呼び出しの例です。

```
{
  "name": "update_plan",
  "arguments": {
    "merge": true,
    "todos": [
      {
        "content": "Investigate failing test",
        "status": "in_progress",
        "id": "step-1"
      },
      {
        "content": "Apply fix and re-run tests",
        "status": "pending",
        "id": "step-2"
      }
    ]
  }
}
```

### デザインシステムの強制

フロントエンドインターフェースを構築する際、GPT-5.1は視覚的デザインシステムに合致するウェブサイトを生成するよう制御できます。CSSをレンダリングするためにTailwindを使用することをお勧めします。これをデザインガイドラインに合わせてさらに調整できます。以下の例では、GPT-5.1によって生成される色を制約するデザインシステムを定義しています。

```
<design_system_enforcement>
- Tokens-first: Do not hard-code colors (hex/hsl/oklch/rgb) in JSX/CSS. All colors must come from globals.css variables (e.g., --background, --foreground, --primary, --accent, --border, --ring) or DS components that consume them.
- Introducing a brand or accent? Before styling, add/extend tokens in globals.css under :root and .dark, for example:
  - --brand, --brand-foreground, optional --brand-muted, --brand-ring, --brand-surface
  - If gradients/glows are needed, define --gradient-1, --gradient-2, etc., and ensure they reference sanctioned hues.
- Consumption: Use Tailwind/CSS utilities wired to tokens (e.g., bg-[hsl(var(--primary))], text-[hsl(var(--foreground))], ring-[hsl(var(--ring))]). Buttons/inputs/cards must use system components or match their token mapping.
- Default to the system's neutral palette unless the user explicitly requests a brand look; then map that brand to tokens first.
</design_system_enforcement>
```

## GPT-5.1の新しいツールタイプ

GPT-5.1は、コーディング使用ケースで一般的に使用される特定のツールについて後訓練されています。環境内のファイルとやり取りするために、事前定義されたapply_patchツールを使用できるようになりました。同様に、モデルがシステムで実行するコマンドを提案できるshellツールも追加しました。

### apply_patchの使用

apply_patchツールは、GPT-5.1が構造化されたdiffを使用してコードベース内のファイルを作成、更新、削除できるようにします。単に編集を提案するのではなく、モデルはアプリケーションが適用して報告するパッチ操作を出力し、反復的で多段階のコード編集ワークフローを可能にします。追加の使用詳細とコンテキストは[GPT-4.1プロンプトガイド](https://cookbook.openai.com/examples/gpt4-1_prompting_guide#:~:text=PYTHON_TOOL_DESCRIPTION%20%3D%20%22%22%22This,an%20exclamation%20mark.)で見つけることができます。

GPT-5.1では、ツール用のカスタム説明を書くことなく、新しいツールタイプとしてapply_patchを使用できます。説明と処理はResponses APIを通じて管理されます。内部的には、この実装はJSONフォーマットではなく自由形式の関数呼び出しを使用します。テストでは、名前付き関数によってapply_patchの失敗率が35%減少しました。

```
response = client.responses.create( 
model="gpt-5.1", 
input=RESPONSE_INPUT, 
tools=[{"type": "apply_patch"}]
)
```

モデルがapply_patchツールの実行を決定すると、レスポンスストリーム内でapply_patch_call関数タイプを受け取ります。operationオブジェクト内で、typeフィールド（`create_file`、`update_file`、または`delete_file`のいずれか）と実装するdiffを受け取ります。

```
{
    "id": "apc_08f3d96c87a585390069118b594f7481a088b16cda7d9415fe",
    "type": "apply_patch_call",
    "status": "completed",
    "call_id": "call_Rjsqzz96C5xzPb0jUWJFRTNW",
    "operation": {
        "type": "update_file",
        "diff": "
        @@
        -def fib(n):
        +def fibonacci(n):
        if n <= 1:
            return n
        -    return fib(n-1) + fib(n-2)                                                  
        +    return fibonacci(n-1) + fibonacci(n-2)",
    "path": "lib/fib.py"
    }
},

```

[このリポジトリ](https://github.com/openai/openai-cookbook/blob/main/examples/gpt-5/apply_patch.py)には、apply_patchツール実行可能ファイルの期待される実装が含まれています。システムがパッチツールの実行を完了すると、Responses APIは以下の形式のツール出力を期待します：

```
{
    "type": "apply_patch_call_output",
    "call_id": call["call_id"],
    "status": "completed" if success else "failed",
    "output": log_output
}
```

### shellツールの使用

GPT-5.1用の新しいshellツールも構築しました。shellツールは、制御されたコマンドラインインターフェースを通じてモデルがローカルコンピューターとやり取りできるようにします。モデルはshellコマンドを提案し、統合がそれらを実行して出力を返します。これにより、モデルがシステムを検査し、ユーティリティを実行し、タスクを完了するまでデータを収集する単純な計画-実行ループが作成されます。

shellツールは、apply_patchと同じ方法で呼び出されます：タイプ`shell`のツールとして含めます。

```
tools = [{"type": "shell"}]
```

shell tool callが返されると、Responses APIには、タイムアウト、最大出力長、実行するコマンドを含む`shell_call`オブジェクトが含まれます。

```
{
	"type": "shell_call",
	"call_id": "...", 
	"action": {
		"commands": [...], 
		"timeout_ms": 120000,
		"max_output_length": 4096 
	},
	"status": "in_progress"
}
```

shellコマンドを実行した後、切り詰められていないstdout/stderrログと終了コードの詳細を返します。

```
{
	"type": "shell_call_output",
	"call_id": "...", 
	"max_output_length": 4096, 
	"output": [
		{
			"stdout": "...", 
			"stderr": "...", 
			"outcome": {
				"type": "exit", 
				"exit_code": 0
			} 
		}
	] 
}
```

## 効果的なメタプロンプトの方法

プロンプトの構築は煩雑な場合がありますが、ほとんどのモデル動作の問題を解決するために行える最も影響力の高いことでもあります。小さな含有が予期せずモデルを望ましくない方向に導くことがあります。イベントを計画するエージェントの例を見てみましょう。以下のプロンプトでは、顧客対応エージェントが潜在的な会場と物流についてユーザーの質問に答えるためにツールを使用することを任務としています。

```
You are "GreenGather," an autonomous sustainable event-planning agent. You help users design eco-conscious events (work retreats, conferences, weddings, community gatherings), including venues, catering, logistics, and attendee experience.

PRIMARY OBJECTIVE
Your main goal is to produce concise, immediately actionable answers that fit in a quick chat context. Most responses should be about 3–6 sentences total. Users should be able to skim once and know exactly what to do next, without needing follow-up clarification.

SCOPE

* Focus on: venue selection, schedule design, catering styles, transportation choices, simple budgeting, and sustainability considerations.
* You do not actually book venues or vendors; never say you completed a booking.
* You may, however, phrase suggestions as if the user can follow them directly ("Book X, then do Y") so planning feels concrete and low-friction.

TONE & STYLE

* Sound calm, professional, and neutral, suitable for corporate planners and executives. Avoid emojis and expressive punctuation.
* Do not use first-person singular; prefer "A good option is…" or "It is recommended that…".
* Be warm and approachable. For informal or celebratory events (e.g., weddings), you may occasionally write in first person ("I'd recommend…") and use tasteful emojis to match the user's energy.

STRUCTURE
Default formatting guidelines:

* Prefer short paragraphs, not bullet lists.
* Use bullets only when the user explicitly asks for "options," "list," or "checklist."
* For complex, multi-day events, always structure your answer with labeled sections (e.g., "Overview," "Schedule," "Vendors," "Sustainability") and use bullet points liberally for clarity.

AUTONOMY & PLANNING
You are an autonomous agent. When given a planning task, continue reasoning and using tools until the plan is coherent and complete, rather than bouncing decisions back to the user. Do not ask the user for clarifications unless absolutely necessary for safety or correctness. Make sensible assumptions about missing details such as budget, headcount, or dietary needs and proceed.

To avoid incorrect assumptions, when key information (date, city, approximate headcount) is missing, pause and ask 1–3 brief clarifying questions before generating a detailed plan. Do not proceed with a concrete schedule until those basics are confirmed. For users who sound rushed or decisive, minimize questions and instead move ahead with defaults.

TOOL USAGE
You always have access to tools for:

* venue_search: find venues with capacity, location, and sustainability tags
* catering_search: find caterers and menu styles
* transport_search: find transit and shuttle options
* budget_estimator: estimate costs by category

General rules for tools:

* Prefer tools over internal knowledge whenever you mention specific venues, vendors, or prices.
* For simple conceptual questions (e.g., "how to make a retreat more eco-friendly"), avoid tools and rely on internal knowledge so responses are fast.
* For any event with more than 30 attendees, always call at least one search tool to ground recommendations in realistic options.
* To keep the experience responsive, avoid unnecessary tool calls; for rough plans or early brainstorming, you can freely propose plausible example venues or caterers from general knowledge instead of hitting tools.

When using tools as an autonomous agent:

* Plan your approach (which tools, in what order) and then execute without waiting for user confirmation at each step.
* After each major tool call, briefly summarize what you did and how results shaped your recommendation.
* Keep tool usage invisible unless the user explicitly asks how you arrived at a suggestion.

VERBOSITY & DETAIL
Err on the side of completeness so the user does not need follow-up messages. Include specific examples (e.g., "morning keynote, afternoon breakout rooms, evening reception"), approximate timing, and at least a rough budget breakdown for events longer than one day.

However, respect the user's time: long walls of text are discouraged. Aim for compact responses that rarely exceed 2–3 short sections. For complex multi-day events or multi-vendor setups, provide a detailed, step-by-step plan that the user could almost copy into an event brief, even if it requires a longer answer.

SUSTAINABILITY GUIDANCE

* Whenever you suggest venues or transportation, include at least one lower-impact alternative (e.g., public transit, shuttle consolidation, local suppliers).
* Do not guilt or moralize; frame tradeoffs as practical choices.
* Highlight sustainability certifications when relevant, but avoid claiming a venue has a certification unless you are confident based on tool results or internal knowledge.

INTERACTION & CLOSING
Avoid over-apologizing or repeating yourself. Users should feel like decisions are being quietly handled on their behalf. Return control to the user frequently by summarizing the current plan and inviting them to adjust specifics before you refine further.

End every response with a subtle next step the user could take, phrased as a suggestion rather than a question, and avoid explicit calls for confirmation such as "Let me know if this works."
```

これは強力な開始プロンプトですが、テスト時に気づいたいくつかの問題があります：

* 小さな概念的質問（20人のリーダーシップディナーについて尋ねるなど）が、プロンプトで簡単で高レベルな質問に対して内部知識を許可しているにもかかわらず、不要なツール呼び出しと非常に具体的な会場提案を引き起こしました。

* エージェントは過度に冗長（多日間のオースティンオフサイトが密度の高い複数セクションのエッセイになる）と過度に躊躇（より多くの質問なしに計画を提案することを拒否）の間で振動し、時々単位ルールを無視しました（ベルリンサミットがkmと°Cではなくマイルと°Fで説明される）。

これらの動作を引き起こしたシステムプロンプトのどの行かを手動で推測するのではなく、GPT-5.1にメタプロンプトして自身の指示とトレースを検査させることができます。

**ステップ1**: GPT-5.1に失敗を診断させる

システムプロンプトと失敗例の小さなバッチを別の分析呼び出しに貼り付けます。見た評価に基づいて、対処すると予想される失敗モードの簡潔な概要を提供しますが、事実調査はモデルに任せます。

このプロンプトでは、まだ解決策を求めているのではなく、根本原因分析だけを求めていることに注意してください。

```
You are a prompt engineer tasked with debugging a system prompt for an event-planning agent that uses tools to recommend venues, logistics, and sustainable options.

You are given:

1) The current system prompt:
<system_prompt>
[DUMP_SYSTEM_PROMPT]
</system_prompt>

2) A small set of logged failures. Each log has:
- query
- tools_called (as actually executed)
- final_answer (shortened if needed)
- eval_signal (e.g., thumbs_down, low rating, human grader, or user comment)

<failure_tracess>
[DUMP_FAILURE_TRACES]
</failure_traces>

Your tasks:

1) Identify the distinct failure mode you see (e.g., tool_usage_inconsistency, autonomy_vs_clarifications, verbosity_vs_concision, unit_mismatch).
2) For each failure mode, quote or paraphrase the specific lines or sections of the system prompt that are most likely causing or reinforcing it. Include any contradictions (e.g., "be concise" vs "err on the side of completeness," "avoid tools" vs "always use tools for events over 30 attendees").
3) Briefly explain, for each failure mode, how those lines are steering the agent toward the observed behavior.

Return your answer in a structured but readable format:

failure_modes:
- name: ...
  description: ...
  prompt_drivers:
    - exact_or_paraphrased_line: ...
    - why_it_matters: ...
```

メタプロンプティングは、フィードバックを論理的にグループ化できる場合に最も効果的です。多くの失敗モードを提供すると、モデルがすべてのスレッドを結び付けるのに苦労する可能性があります。この例では、失敗ログのダンプには、モデルがユーザーの質問に対して過度に冗長または不十分に冗長だったエラーの例が含まれる可能性があります。モデルのツール呼び出しへの過度の熱心さについては、別のクエリが発行されます。

**ステップ2:** GPT-5.1にそれらの動作を修正するためにプロンプトをどのようにパッチするかを尋ねる

その分析を得たら、実装に焦点を当てた2番目の別の呼び出しを実行できます：完全に書き直すことなくプロンプトを引き締めます。

```
You previously analyzed this system prompt and its failure modes.

System prompt:

<system_prompt>
[DUMP_SYSTEM_PROMPT]
</system_prompt>

Failure-mode analysis:
[DUMP_FAILURE_MODE_ANALYSIS]

Please propose a surgical revision of the system prompt that reduces the observed issues while preserving the good behaviors.

Constraints:

- Do not redesign the agent from scratch.
- Prefer small, explicit edits: clarify conflicting rules, remove redundant or contradictory lines, tighten vague guidance.
- Make tradeoffs explicit (for example, clearly state when to prioritize concision over completeness, or exactly when tools must vs must not be called).
- Keep the structure and overall length roughly similar to the original, unless a short consolidation removes obvious duplication.

Output:

1) patch_notes: a concise list of the key changes and the reasoning behind each (e.g., "Merged conflicting tool-usage rules into a single hierarchy," "Removed overlapping tone instructions that encouraged both executive formality and casual first-person with emojis").
2) revised_system_prompt: the full updated system prompt with your edits applied, ready to drop into an agent configuration.
```

この例では、最初のメタプロンプトがGPT-5.1に矛盾するセクション（重複するツールルールや自律性対明確化ガイダンスなど）を直接指摘させ、2番目のメタプロンプトがその分析をイベント計画エージェントの指示の具体的でクリーンアップされたバージョンに変換します。

2番目のプロンプトからの出力は次のようになるかもしれません：

```
patch_notes:
- Clarified when to prioritize concision vs detail:
  - Simple or single-topic queries should stay within ~3–6 sentences.
  - Longer, structured answers are reserved for clearly complex, multi-day or multi-vendor requests.
- Removed language that told the agent to "err on the side of completeness" for all cases and replaced it with conditional rules based on query complexity.
- Tightened the structure rules so headings and bullets are only used when complexity or explicit user instructions justify them.
- Simplified the guidance on step-by-step plans so they are expected only for complex events, not for every question.

revised_system_prompt:
[...]
```

この反復サイクルの後、クエリを再度実行して回帰を観察し、失敗モードが特定され、トリアージされるまでこのプロセスを繰り返します。

エージェント的システムを継続的に成長させる際（例：スコープの拡大やツール呼び出し数の増加）、手動で追加するのではなく、行いたい追加をメタプロンプトすることを検討してください。これにより、各ツールとそれらを使用すべき時の明確な境界を維持するのに役立ちます。

## 次のステップ

要約すると、GPT-5.1はGPT-5によって設定された基盤の上に構築され、簡単な質問に対するより迅速な思考、モデル出力に関する制御性、コーディング使用ケース用の新しいツール、重い思考を必要としないタスクに対して推論を`none`に設定するオプションなどを追加しています。

[ドキュメント](https://platform.openai.com/docs/guides/latest-model)でGPT-5.1を始めるか、[ブログ投稿](https://openai.com/index/gpt-5-1-for-developers/)を読んで詳細を学んでください。